In [43]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
import datetime

API_KEY = "PK1W6UGQYV6DJYD9HEP7"
SECRET_KEY = "ndH61mpUAGkK7jNKM8fgl1EvjUQKf0yDofyYP0pe"

client = CryptoHistoricalDataClient()
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD", 'ETH/USD', 'USDT/USD', 'XRP/USD', 'BNB/USD', 'SOL/USD', 'USDC/USD', 'TRX/USD', 'DOGE/USD', 'ADA/USD'],
  timeframe=TimeFrame.Day,
  start=datetime.datetime(2020, 7, 1),
  end=datetime.datetime(2025, 7, 1)
)

btc_bars = client.get_crypto_bars(request_params)

df = btc_bars.df

In [44]:
import pandas as pd

ts = df.index.get_level_values('timestamp')
ts = pd.to_datetime(ts)

dow_dummies = pd.get_dummies(ts.day_name(), prefix='is', dtype=bool).reindex(
    columns=['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday',
             'is_friday', 'is_saturday', 'is_sunday'], fill_value=False)

def season(month):
    if month in (12, 1, 2):
        return 'winter'
    elif month in (3, 4, 5):
        return 'spring'
    elif month in (6, 7, 8):
        return 'summer'
    else:
        return 'autumn'

season_dummies = pd.get_dummies([season(m) for m in ts.month], prefix='is', dtype=bool).reindex(
    columns=['is_winter', 'is_spring', 'is_summer', 'is_autumn'], fill_value=False)

dow_dummies.index = df.index
season_dummies.index = df.index

df = df.join(dow_dummies).join(season_dummies)

In [45]:
import requests

def get_fear_greed_data():
    url = "https://api.alternative.me/fng/?limit=0&format=json"
    response = requests.get(url)
    data = response.json()['data']
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s').dt.date  # только дата
    df['fear&greed'] = df['value'].astype(int)
    return df[['timestamp', 'fear&greed']]

fng_df = get_fear_greed_data()

df = df.reset_index()
df['date'] = df['timestamp'].dt.date

df = df.merge(fng_df, how='left', left_on='date', right_on='timestamp')

df = df.drop(columns=['timestamp_y', 'date'])
df = df.rename(columns={'timestamp_x': 'timestamp'})

df = df.set_index(['symbol', 'timestamp'])

In [46]:
df

open      high       low     close  \
symbol  timestamp                                                           
TRX/USD 2022-03-31 05:00:00+00:00    0.2500    0.2500    0.0711    0.0731   
        2022-04-01 05:00:00+00:00    0.0731    0.0764    0.0714    0.0760   
        2022-04-02 05:00:00+00:00    0.0759    0.0764    0.0720    0.0732   
        2022-04-03 05:00:00+00:00    0.0732    0.0738    0.0717    0.0717   
        2022-04-04 05:00:00+00:00    0.0718    0.0723    0.0688    0.0717   
...                                     ...       ...       ...       ...   
SOL/USD 2025-06-26 05:00:00+00:00  145.4250  145.9560  137.2205  141.4300   
        2025-06-27 05:00:00+00:00  141.3715  144.8450  140.1890  143.3550   
        2025-06-28 05:00:00+00:00  143.3640  152.7500  143.3550  149.8430   
        2025-06-29 05:00:00+00:00  149.8000  154.7535  149.4000  151.0485   
        2025-06-30 05:00:00+00:00  151.0000  159.9595  149.5050  152.5125   

                                         volume  trade_count        vwap  \
symbol  timestamp                                                          
TRX/USD 2022-03-31 05:00:00+00:00  4.922430e+07       4722.0    0.083005   
        2022-04-01 05:00:00+00:00  1.806861e+07       2104.0    0.074022   
        2022-04-02 05:00:00+00:00  8.419898e+06        958.0    0.074250   
        2022-04-03 05:00:00+00:00  3.998644e+06        634.0    0.072536   
        2022-04-04 05:00:00+00:00  4.657212e+06        643.0    0.070850   
...                                         ...          ...         ...   
SOL/USD 2025-06-26 05:00:00+00:00  2.316496e+02        282.0  141.885314   
        2025-06-27 05:00:00+00:00  7.399703e+01         44.0  142.446254   
        2025-06-28 05:00:00+00:00  1.250214e+02        146.0  150.616842   
        2025-06-29 05:00:00+00:00  1.726026e+02        104.0  151.646053   
        2025-06-30 05:00:00+00:00  2.139907e+02        118.0  154.699590   

                                   is_monday  is_tuesday  is_wednesday  \
symbol  timestamp                                                        
TRX/USD 2022-03-31 05:00:00+00:00      False       False         False   
        2022-04-01 05:00:00+00:00      False       False         False   
        2022-04-02 05:00:00+00:00      False       False         False   
        2022-04-03 05:00:00+00:00      False       False         False   
        2022-04-04 05:00:00+00:00      False       False         False   
...                                      ...         ...           ...   
SOL/USD 2025-06-26 05:00:00+00:00      False       False         False   
        2025-06-27 05:00:00+00:00      False       False         False   
        2025-06-28 05:00:00+00:00      False       False         False   
        2025-06-29 05:00:00+00:00      False       False         False   
        2025-06-30 05:00:00+00:00      False       False         False   

                                   is_thursday  is_friday  is_saturday  \
symbol  timestamp                                                        
TRX/USD 2022-03-31 05:00:00+00:00        False      False        False   
        2022-04-01 05:00:00+00:00        False      False        False   
        2022-04-02 05:00:00+00:00        False      False        False   
        2022-04-03 05:00:00+00:00        False      False        False   
        2022-04-04 05:00:00+00:00        False      False        False   
...                                        ...        ...          ...   
SOL/USD 2025-06-26 05:00:00+00:00        False      False        False   
        2025-06-27 05:00:00+00:00        False      False        False   
        2025-06-28 05:00:00+00:00        False      False        False   
        2025-06-29 05:00:00+00:00        False      False        False   
        2025-06-30 05:00:00+00:00        False      False        False   

                                   is_sunday  is_winter  is_spring  is_summer  \
symbol  timestamp                        